In [ ]:
!python -m spacy download pt_core_news_sm

In [ ]:
!pip install emoji --upgrade

In [3]:
import pandas as pd
import numpy as np

#utils
import os, random, time

#NLP
import spacy
nlp_pt = spacy.load('pt_core_news_sm')
import nltk
import emoji

In [4]:
emojis_list = list(emoji.UNICODE_EMOJI['pt'].keys())

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
reviews_df = pd.read_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/amazon_reviews_scraping/reviews.csv')
reviews_df

,Title,Rating,Comment,Date Review
0,\n\n\n\n\n\n\n\n \n \n Decepcionado\n \n,"1,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom, em qu...",Avaliado no Brasil em 9 de junho de 2021
1,\n\n\n\n\n\n\n\n \n \n Reclamação não ate...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n O produto ...,Avaliado no Brasil em 26 de novembro de 2020
2,\n\n\n\n\n\n\n\n \n \n faltou a informaçã...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Quem compr...,Avaliado no Brasil em 5 de janeiro de 2021
3,\n\n\n\n\n\n\n\n \n \n Ótimo para iniciar...,"3,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n A funciona...,Avaliado no Brasil em 16 de junho de 2021
4,\n\n\n\n\n\n\n\n \n \n Entendi alguns can...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Excelente ...,Avaliado no Brasil em 10 de dezembro de 2020
...,...,...,...,...
3805,\n\n\n\n\n\n\n\n \n \n Recomendo\n \n,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n ...,Avaliado no Brasil em 6 de julho de 2021
3806,\n\n\n\n\n\n\n\n \n \n Cabo da tomada de ...,"4,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom dia, c...",Avaliado no Brasil em 4 de maio de 2021
3807,\n\n\n\n\n\n\n\n \n \n Ótima assistente\n...,"4,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Amei a Ale...,Avaliado no Brasil em 18 de maio de 2021
3808,\n\n\n\n\n\n\n\n \n \n Excelente produto\...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Atendeu mi...,Avaliado no Brasil em 9 de janeiro de 2021


# Pré processamento das notas

In [7]:
reviews_df['int_rating'] = reviews_df['Rating'].apply(lambda x: int(x[0]))
reviews_df['sent_rating'] = reviews_df['int_rating'].apply(lambda x: 'positivo' if x > 3 else 'negativo')
#reviews_df['sent_rating'] = reviews_df['int_rating'].apply(lambda x: 'positivo' if x > 3 else 'neutro' if x == 3 else 'negativo')
reviews_df

,Title,Rating,Comment,Date Review,int_rating,sent_rating
0,\n\n\n\n\n\n\n\n \n \n Decepcionado\n \n,"1,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom, em qu...",Avaliado no Brasil em 9 de junho de 2021,1,negativo
1,\n\n\n\n\n\n\n\n \n \n Reclamação não ate...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n O produto ...,Avaliado no Brasil em 26 de novembro de 2020,1,negativo
2,\n\n\n\n\n\n\n\n \n \n faltou a informaçã...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Quem compr...,Avaliado no Brasil em 5 de janeiro de 2021,1,negativo
3,\n\n\n\n\n\n\n\n \n \n Ótimo para iniciar...,"3,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n A funciona...,Avaliado no Brasil em 16 de junho de 2021,3,negativo
4,\n\n\n\n\n\n\n\n \n \n Entendi alguns can...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Excelente ...,Avaliado no Brasil em 10 de dezembro de 2020,5,positivo
...,...,...,...,...,...,...
3805,\n\n\n\n\n\n\n\n \n \n Recomendo\n \n,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n ...,Avaliado no Brasil em 6 de julho de 2021,5,positivo
3806,\n\n\n\n\n\n\n\n \n \n Cabo da tomada de ...,"4,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom dia, c...",Avaliado no Brasil em 4 de maio de 2021,4,positivo
3807,\n\n\n\n\n\n\n\n \n \n Ótima assistente\n...,"4,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Amei a Ale...,Avaliado no Brasil em 18 de maio de 2021,4,positivo
3808,\n\n\n\n\n\n\n\n \n \n Excelente produto\...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Atendeu mi...,Avaliado no Brasil em 9 de janeiro de 2021,5,positivo


# Pré-processamento das datas

In [8]:
import datetime

reviews_df['data'] = 0

meses = {'janeiro':1, 'fevereiro':2, 'março':3, 'abril':4, 'maio':5, 'junho':6, 
         'julho':7, 'agosto':8, 'setembro':9, 'outubro':10, 'novembro':11, 'dezembro':12}
for i, data in enumerate(reviews_df['Date Review']):
  data_list = data.split(' ')
  dia = int(data_list[4])
  mes = meses[data_list[6]]
  ano = int(data_list[8])
  data_form = datetime.datetime(ano, mes, dia)
  reviews_df.loc[i,'data'] = data_form

reviews_df

,Title,Rating,Comment,Date Review,int_rating,sent_rating,data
0,\n\n\n\n\n\n\n\n \n \n Decepcionado\n \n,"1,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom, em qu...",Avaliado no Brasil em 9 de junho de 2021,1,negativo,2021-06-09 00:00:00
1,\n\n\n\n\n\n\n\n \n \n Reclamação não ate...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n O produto ...,Avaliado no Brasil em 26 de novembro de 2020,1,negativo,2020-11-26 00:00:00
2,\n\n\n\n\n\n\n\n \n \n faltou a informaçã...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Quem compr...,Avaliado no Brasil em 5 de janeiro de 2021,1,negativo,2021-01-05 00:00:00
3,\n\n\n\n\n\n\n\n \n \n Ótimo para iniciar...,"3,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n A funciona...,Avaliado no Brasil em 16 de junho de 2021,3,negativo,2021-06-16 00:00:00
4,\n\n\n\n\n\n\n\n \n \n Entendi alguns can...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Excelente ...,Avaliado no Brasil em 10 de dezembro de 2020,5,positivo,2020-12-10 00:00:00
...,...,...,...,...,...,...,...
3805,\n\n\n\n\n\n\n\n \n \n Recomendo\n \n,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n ...,Avaliado no Brasil em 6 de julho de 2021,5,positivo,2021-07-06 00:00:00
3806,\n\n\n\n\n\n\n\n \n \n Cabo da tomada de ...,"4,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom dia, c...",Avaliado no Brasil em 4 de maio de 2021,4,positivo,2021-05-04 00:00:00
3807,\n\n\n\n\n\n\n\n \n \n Ótima assistente\n...,"4,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Amei a Ale...,Avaliado no Brasil em 18 de maio de 2021,4,positivo,2021-05-18 00:00:00
3808,\n\n\n\n\n\n\n\n \n \n Excelente produto\...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Atendeu mi...,Avaliado no Brasil em 9 de janeiro de 2021,5,positivo,2021-01-09 00:00:00


# Processamento sem Stemming

## Remoção de todas stopwords, emojis e pontuações

In [9]:
def tokenizer(text,stopwords):

  new_string = ''
  text = text.strip()
  text = ' '.join(text.split())
  doc = nlp_pt(text)
  sentences = doc.sents

  for sent in sentences:
    tokens = nlp_pt(sent.text)
    for token in tokens:
      if token.is_punct == False and token.text.lower() not in stopwords and token.text not in emojis_list:
        new_string = new_string + ' ' + token.text

  return new_string.strip()


In [10]:
stopwords_full = nlp_pt.Defaults.stop_words

reviews_df['processed_text_NoStopwords'] = reviews_df['Comment'].apply(lambda x: tokenizer(x,stopwords_full))
reviews_df

,Title,Rating,Comment,Date Review,int_rating,sent_rating,data,processed_text_NoStopwords
0,\n\n\n\n\n\n\n\n \n \n Decepcionado\n \n,"1,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom, em qu...",Avaliado no Brasil em 9 de junho de 2021,1,negativo,2021-06-09 00:00:00,som realmente entanto conectividade resposta m...
1,\n\n\n\n\n\n\n\n \n \n Reclamação não ate...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n O produto ...,Avaliado no Brasil em 26 de novembro de 2020,1,negativo,2020-11-26 00:00:00,produto responde corretamente comandos preciso...
2,\n\n\n\n\n\n\n\n \n \n faltou a informaçã...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Quem compr...,Avaliado no Brasil em 5 de janeiro de 2021,1,negativo,2021-01-05 00:00:00,compra caixa som pensa normalmente leva-la Ale...
3,\n\n\n\n\n\n\n\n \n \n Ótimo para iniciar...,"3,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n A funciona...,Avaliado no Brasil em 16 de junho de 2021,3,negativo,2021-06-16 00:00:00,funcionalidade Comprei lâmpadas smart plugs co...
4,\n\n\n\n\n\n\n\n \n \n Entendi alguns can...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Excelente ...,Avaliado no Brasil em 10 de dezembro de 2020,5,positivo,2020-12-10 00:00:00,Excelente assistente som limpo Alexa dificulda...
...,...,...,...,...,...,...,...,...
3805,\n\n\n\n\n\n\n\n \n \n Recomendo\n \n,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n ...,Avaliado no Brasil em 6 de julho de 2021,5,positivo,2021-07-06 00:00:00,mídia carregada Gostei comprei impulso arrepen...
3806,\n\n\n\n\n\n\n\n \n \n Cabo da tomada de ...,"4,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom dia, c...",Avaliado no Brasil em 4 de maio de 2021,4,positivo,2021-05-04 00:00:00,dia comprei echo dot preta cabo tomada veio br...
3807,\n\n\n\n\n\n\n\n \n \n Ótima assistente\n...,"4,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Amei a Ale...,Avaliado no Brasil em 18 de maio de 2021,4,positivo,2021-05-18 00:00:00,Amei Alexa incrível fiquei chateada descobri f...
3808,\n\n\n\n\n\n\n\n \n \n Excelente produto\...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Atendeu mi...,Avaliado no Brasil em 9 de janeiro de 2021,5,positivo,2021-01-09 00:00:00,Atendeu expectativas Reconhece voz entende per...


In [11]:
NoStopwords_df = reviews_df[['sent_rating', 'processed_text_NoStopwords','data']]
NoStopwords_df.to_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/datasets/NoStem_TotalStopwordRemoval_dataset.csv',index=False)


## Stopwords mantidas, pontuações e emojis removidos

In [12]:
def tokenizer2(text):

  new_string = ''
  text = text.strip()
  text = ' '.join(text.split())
  doc = nlp_pt(text)
  sentences = doc.sents

  for sent in sentences:
    tokens = nlp_pt(sent.text)
    for token in tokens:
      if token.is_punct == False and token.text not in emojis_list:
        new_string = new_string + ' ' + token.text

  return new_string.strip()

In [13]:
reviews_df['processed_text_KeepStopwords'] = reviews_df['Comment'].apply(lambda x: tokenizer2(x))
reviews_df

,Title,Rating,Comment,Date Review,int_rating,sent_rating,data,processed_text_NoStopwords,processed_text_KeepStopwords
0,\n\n\n\n\n\n\n\n \n \n Decepcionado\n \n,"1,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom, em qu...",Avaliado no Brasil em 9 de junho de 2021,1,negativo,2021-06-09 00:00:00,som realmente entanto conectividade resposta m...,Bom em questão de som ela é realmente boa no e...
1,\n\n\n\n\n\n\n\n \n \n Reclamação não ate...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n O produto ...,Avaliado no Brasil em 26 de novembro de 2020,1,negativo,2020-11-26 00:00:00,produto responde corretamente comandos preciso...,O produto não responde corretamente aos comand...
2,\n\n\n\n\n\n\n\n \n \n faltou a informaçã...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Quem compr...,Avaliado no Brasil em 5 de janeiro de 2021,1,negativo,2021-01-05 00:00:00,compra caixa som pensa normalmente leva-la Ale...,Quem compra uma caixa de som pensa normalmente...
3,\n\n\n\n\n\n\n\n \n \n Ótimo para iniciar...,"3,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n A funciona...,Avaliado no Brasil em 16 de junho de 2021,3,negativo,2021-06-16 00:00:00,funcionalidade Comprei lâmpadas smart plugs co...,A funcionalidade é muito boa Comprei as lâmpad...
4,\n\n\n\n\n\n\n\n \n \n Entendi alguns can...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Excelente ...,Avaliado no Brasil em 10 de dezembro de 2020,5,positivo,2020-12-10 00:00:00,Excelente assistente som limpo Alexa dificulda...,Excelente assistente som muito bom e limpo a A...
...,...,...,...,...,...,...,...,...,...
3805,\n\n\n\n\n\n\n\n \n \n Recomendo\n \n,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n ...,Avaliado no Brasil em 6 de julho de 2021,5,positivo,2021-07-06 00:00:00,mídia carregada Gostei comprei impulso arrepen...,A mídia não pôde ser carregada Gostei de tudo ...
3806,\n\n\n\n\n\n\n\n \n \n Cabo da tomada de ...,"4,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom dia, c...",Avaliado no Brasil em 4 de maio de 2021,4,positivo,2021-05-04 00:00:00,dia comprei echo dot preta cabo tomada veio br...,Bom dia comprei a echo dot preta e o cabo da t...
3807,\n\n\n\n\n\n\n\n \n \n Ótima assistente\n...,"4,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Amei a Ale...,Avaliado no Brasil em 18 de maio de 2021,4,positivo,2021-05-18 00:00:00,Amei Alexa incrível fiquei chateada descobri f...,Amei a Alexa ela é incrível Porém fiquei chate...
3808,\n\n\n\n\n\n\n\n \n \n Excelente produto\...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Atendeu mi...,Avaliado no Brasil em 9 de janeiro de 2021,5,positivo,2021-01-09 00:00:00,Atendeu expectativas Reconhece voz entende per...,Atendeu minhas expectativas Reconhece voz ente...


In [14]:
KeepStopwords_df = reviews_df[['sent_rating', 'processed_text_KeepStopwords','data']]
KeepStopwords_df.to_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/datasets/NoStem_StopwordKeep_dataset.csv',index=False)


# Processamento com Stemming

## Remoção de todas stopwords, emojis e pontuações

In [15]:
def tokenizer_stem(text,stopwords,stemmer):

  new_string = ''
  text = text.strip()
  text = ' '.join(text.split())
  doc = nlp_pt(text)
  sentences = doc.sents

  for sent in sentences:
    tokens = nlp_pt(sent.text)
    for token in tokens:
      if token.is_punct == False and token.text.lower() not in stopwords and token.text not in emojis_list:
        stem = stemmer.stem(token.text)
        new_string = new_string + ' ' + stem

  return new_string.strip()

In [16]:
stemmer = nltk.stem.porter.PorterStemmer()

stopwords = nlp_pt.Defaults.stop_words

reviews_df['processed_text_stem_NoStopwords'] = reviews_df['Comment'].apply(lambda x: tokenizer_stem(x,stopwords_full,stemmer))
reviews_df

,Title,Rating,Comment,Date Review,int_rating,sent_rating,data,processed_text_NoStopwords,processed_text_KeepStopwords,processed_text_stem_NoStopwords
0,\n\n\n\n\n\n\n\n \n \n Decepcionado\n \n,"1,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom, em qu...",Avaliado no Brasil em 9 de junho de 2021,1,negativo,2021-06-09 00:00:00,som realmente entanto conectividade resposta m...,Bom em questão de som ela é realmente boa no e...,som realment entanto conectividad resposta mic...
1,\n\n\n\n\n\n\n\n \n \n Reclamação não ate...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n O produto ...,Avaliado no Brasil em 26 de novembro de 2020,1,negativo,2020-11-26 00:00:00,produto responde corretamente comandos preciso...,O produto não responde corretamente aos comand...,produto respond corretament comando preciso mé...
2,\n\n\n\n\n\n\n\n \n \n faltou a informaçã...,"1,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Quem compr...,Avaliado no Brasil em 5 de janeiro de 2021,1,negativo,2021-01-05 00:00:00,compra caixa som pensa normalmente leva-la Ale...,Quem compra uma caixa de som pensa normalmente...,compra caixa som pensa normalment leva-la alex...
3,\n\n\n\n\n\n\n\n \n \n Ótimo para iniciar...,"3,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n A funciona...,Avaliado no Brasil em 16 de junho de 2021,3,negativo,2021-06-16 00:00:00,funcionalidade Comprei lâmpadas smart plugs co...,A funcionalidade é muito boa Comprei as lâmpad...,funcionalidad comprei lâmpada smart plug contr...
4,\n\n\n\n\n\n\n\n \n \n Entendi alguns can...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Excelente ...,Avaliado no Brasil em 10 de dezembro de 2020,5,positivo,2020-12-10 00:00:00,Excelente assistente som limpo Alexa dificulda...,Excelente assistente som muito bom e limpo a A...,excelent assistent som limpo alexa dificuldad ...
...,...,...,...,...,...,...,...,...,...,...
3805,\n\n\n\n\n\n\n\n \n \n Recomendo\n \n,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n ...,Avaliado no Brasil em 6 de julho de 2021,5,positivo,2021-07-06 00:00:00,mídia carregada Gostei comprei impulso arrepen...,A mídia não pôde ser carregada Gostei de tudo ...,mídia carregada gostei comprei impulso arrepen...
3806,\n\n\n\n\n\n\n\n \n \n Cabo da tomada de ...,"4,0 de 5 estrelas","\n\n\n\n\n\n\n\n\n\n \n \n \n Bom dia, c...",Avaliado no Brasil em 4 de maio de 2021,4,positivo,2021-05-04 00:00:00,dia comprei echo dot preta cabo tomada veio br...,Bom dia comprei a echo dot preta e o cabo da t...,dia comprei echo dot preta cabo tomada veio br...
3807,\n\n\n\n\n\n\n\n \n \n Ótima assistente\n...,"4,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Amei a Ale...,Avaliado no Brasil em 18 de maio de 2021,4,positivo,2021-05-18 00:00:00,Amei Alexa incrível fiquei chateada descobri f...,Amei a Alexa ela é incrível Porém fiquei chate...,amei alexa incrível fiquei chateada descobri f...
3808,\n\n\n\n\n\n\n\n \n \n Excelente produto\...,"5,0 de 5 estrelas",\n\n\n\n\n\n\n\n\n\n \n \n \n Atendeu mi...,Avaliado no Brasil em 9 de janeiro de 2021,5,positivo,2021-01-09 00:00:00,Atendeu expectativas Reconhece voz entende per...,Atendeu minhas expectativas Reconhece voz ente...,atendeu expectativa reconhec voz entend perfei...


In [17]:
Stem_NoStopwords_df = reviews_df[['sent_rating', 'processed_text_stem_NoStopwords','data']]
Stem_NoStopwords_df.to_csv('/content/drive/MyDrive/ProjetoFinal_ProcessText/datasets/Stem_TotalStopwordRemoval_dataset.csv',index=False)
